In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm.notebook import tqdm
from time import sleep

### Gradscope Processor
Used to upload submissions to Gradescope for grading and feedback

#### Plan

1. Manually create courses, assignments (batches) and upload rosters
2. Automate uploading of submission zip files


#### Define assignment (batch) URLs and batches to use

In [ ]:
ASSIGNMENT_URLS = ['https://www.gradescope.com/courses/742074/assignments/4141688/submissions',
                   'https://www.gradescope.com/courses/742075/assignments/4141689/submissions',
                   'https://www.gradescope.com/courses/742076/assignments/4141690/submissions',
                   'https://www.gradescope.com/courses/742077/assignments/4141692/submissions']
BATCH_PATH = '../data/batches/7'

#### Setup Selenium

In [ ]:
browser = webdriver.Firefox()
browser.get(ASSIGNMENT_URLS[0])
# Manually sign in

#### For each zip file upload to Gradescope for each URL

In [ ]:
def upload_batch(assignment_url):
    for file_name in tqdm(os.listdir(BATCH_PATH)):
        if file_name.endswith('.zip'):
            # Load assignment submissions page
            browser.get(assignment_url)

            # Open upload modal
            upload_btn_icon = browser.find_element(By.CLASS_NAME, 'fa-upload')
            upload_btn_icon.find_element(By.XPATH, './..').click()

            # Add Zip File
            file_input = browser.find_element(By.CSS_SELECTOR, "input[type='file']")
            file_input.send_keys(os.path.abspath(os.path.join(BATCH_PATH, file_name)))

            # Add student name
            student_id = file_name.split('_')[2].replace('.zip', '')
            student_name_txt_box = browser.find_element(By.CLASS_NAME, 'studentPicker')\
                .find_element(By.CLASS_NAME, 'selectize-input ').find_element(By.TAG_NAME, 'input')
            student_name_txt_box.send_keys(student_id)

            sleep(1)

            student_name_txt_box.send_keys(Keys.RETURN)

            # Upload
            browser.find_element(By.CLASS_NAME, "js-submitCode").click()

            sleep(5)

In [ ]:
# MAKE SURE THE WINDOW IS NOT SELECTED
for assignment in tqdm(ASSIGNMENT_URLS, total=len(ASSIGNMENT_URLS)):
    upload_batch(assignment)